## 使用决策树(未剪枝)建立手写数字集分类模型
### 正确率88.09%

In [35]:
import numpy as np
import math 
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
trainData = pd.read_csv("trainData.csv")
trainimageArr  = np.array(trainData.iloc[:,1:])
trainLabelArr = np.array(trainData.iloc[:,0])
testDataArr = np.array(pd.read_csv("testData.csv").iloc[:,1:])
testLabelArr = np.array(pd.read_csv("testData.csv").iloc[:,0])
trainimageArr.shape

(60000, 784)

### 使用决策树构建MNIST手写数字集分类器

#### 对于决策书而言,通常我们需要从根节点开始,基于某种特征筛选进行结点的扩展，那么如何选择合适的切分点便成了需要解决的问题
#### 在决策树中引入了熵的概念: ${H(p)=\sum_{i=1}^{M}{-p_i*log(p_i)}}$
#### 熵的物理意义即用于表示一个物体的稳定程度，通常而言熵越大其稳定性越差，熵越小其稳定性越好
#### 此外决策树又引入了信息增益，$g(D,A)=H(D)-H(D|A)$ 其表示的含义以A为划分点的条件下,系统的区分度，通常取增益最大的那个特征作为划分点,然后对于划分过后的数据集进行再次的特征划分点选择
#### $g(D,A)=H(D)-H(D|A)=\sum_{i=1}^{K}-\frac{|D_i|}{|D|}*log(\frac{|D_i|}{|D|}) - \sum_{i=1}^{K}\frac{|D_i|}{|D|}*\sum_{j=1}^{n}\frac{|c_ij|}{|D_i|}*log(\frac{|c_ij|}{|D_i|})$ 其中K表示为特征点取值的总类，n表示为标签所可以取得得总类

In [37]:
def HD(trainlableArr):
    ClassDic = set([label for label in trainlableArr])
    H_D= 0
    for i in ClassDic:
        p=trainlableArr[trainlableArr==i].size/trainlableArr.shape[0]
        H_D+=-p*np.log2(p)
    return H_D

def HD_A(trainDataArr_DevByFeature,trainlabelArr):
    H_D_A = 0
    ClassSet =  set([label for label in trainDataArr_DevByFeature])
    for i in ClassSet:
        H_D_A+=trainDataArr_DevByFeature[trainDataArr_DevByFeature==i].size / trainDataArr_DevByFeature.size*HD(trainlabelArr[trainDataArr_DevByFeature==i])
    return H_D_A

def findMaxFeature(trainDataArr,trainlableArr):
    MaxGDA = -1
    Bestdiv = -1
    H_D = HD(trainlableArr)
    featureNum = trainDataArr.shape[1]
    for i  in range(featureNum):
        trainDevFeature = trainDataArr[:,i].flatten()
        H_D_A = HD_A(trainDevFeature,trainlableArr) 
        G_D_A = H_D - H_D_A
        if G_D_A>MaxGDA:
            MaxGDA=G_D_A
            Bestdiv = i
    return Bestdiv,MaxGDA

def majorClass(labelArr):
    '''
    找到当前标签集中占数目最大的标签
    :param labelArr: 标签集
    :return: 最大的标签
    '''
    #建立字典，用于不同类别的标签技术
    classDict = {}
    #遍历所有标签
    for i in range(len(labelArr)):
        #当第一次遇到A标签时，字典内还没有A标签，这时候直接幅值加1是错误的，
        #所以需要判断字典中是否有该键，没有则创建，有就直接自增
        if labelArr[i] in classDict.keys():
            # 若在字典中存在该标签，则直接加1
            classDict[labelArr[i]] += 1
        else:
            #若无该标签，设初值为1，表示出现了1次了
            classDict[labelArr[i]] = 1
    #对字典依据值进行降序排序
    classSort = sorted(classDict.items(), key=lambda x: x[1], reverse=True)
    #返回最大一项的标签，即占数目最多的标签
    return classSort[0][0]


In [58]:
def subTrainData(trainDataArr,trainLabelArr,div,index):
    newtrainDataArr = trainDataArr[trainDataArr[:,div]==index]
    newtrainLabelArr = trainLabelArr[trainDataArr[:,div]==index]
    newtrainDataArr= np.delete(newtrainDataArr,div,axis=1)
    return newtrainDataArr,newtrainLabelArr
    
def build_tree(trainDataArr, trainLabelArr):
    epsilon = 0.1 ## 最小信息增益
    featureNum = trainDataArr.shape[1]
    print("start a node",featureNum,trainDataArr.shape[0])
    classGroup = set(trainLabelArr)
    if len(classGroup)==1: ##若只剩一类，则为叶子结点，返回
        return trainLabelArr[0]
    if trainDataArr.shape[0]==1: ## 只剩一个叶子结点 则返回
        return trainLabelArr[0]
    div,maxgda = findMaxFeature(trainDataArr,trainLabelArr)
    if maxgda<epsilon:# 若信息增益小于一定的阈值 则返回类别中数量最多的一项
        return majorClass(trainLabelArr.tolist())
    tree ={div:{}}
    tree[div][0] = build_tree(*subTrainData(trainDataArr,trainLabelArr,div,0))
    tree[div][1] = build_tree(*subTrainData(trainDataArr,trainLabelArr,div,1)) 
    return tree

In [59]:
tree = build_tree(trainimageArr,trainLabelArr)

start a node 784 60000
start a node 783 23438
start a node 782 12033
start a node 781 5513
start a node 780 4196
start a node 779 3988
start a node 778 3795
start a node 778 193
start a node 777 70
start a node 776 49
start a node 775 33
start a node 774 19
start a node 773 17
start a node 772 14
start a node 772 3
start a node 771 2
start a node 770 1
start a node 770 1
start a node 771 1
start a node 773 2
start a node 774 14
start a node 773 4
start a node 773 10
start a node 772 9
start a node 772 1
start a node 775 16
start a node 774 11
start a node 774 5
start a node 773 3
start a node 772 2
start a node 772 1
start a node 773 2
start a node 776 21
start a node 775 17
start a node 775 4
start a node 774 2
start a node 773 1
start a node 773 1
start a node 774 2
start a node 777 123
start a node 776 30
start a node 775 7
start a node 774 5
start a node 774 2
start a node 773 1
start a node 773 1
start a node 775 23
start a node 774 21
start a node 773 20
start a node 773 1
start 

start a node 778 564
start a node 777 476
start a node 776 158
start a node 775 142
start a node 774 28
start a node 773 6
start a node 772 2
start a node 771 1
start a node 771 1
start a node 772 4
start a node 773 22
start a node 772 6
start a node 771 3
start a node 771 3
start a node 772 16
start a node 774 114
start a node 775 16
start a node 774 5
start a node 774 11
start a node 773 10
start a node 773 1
start a node 776 318
start a node 775 276
start a node 774 163
start a node 773 155
start a node 772 18
start a node 771 12
start a node 770 10
start a node 769 9
start a node 769 1
start a node 770 2
start a node 771 6
start a node 770 5
start a node 770 1
start a node 772 137
start a node 773 8
start a node 772 4
start a node 771 2
start a node 770 1
start a node 770 1
start a node 771 2
start a node 772 4
start a node 771 2
start a node 771 2
start a node 774 113
start a node 773 51
start a node 772 19
start a node 771 8
start a node 770 7
start a node 770 1
start a node 771 

start a node 774 21
start a node 773 16
start a node 772 13
start a node 772 3
start a node 773 5
start a node 774 171
start a node 775 53
start a node 774 42
start a node 773 13
start a node 772 4
start a node 771 2
start a node 770 1
start a node 770 1
start a node 771 2
start a node 772 9
start a node 773 29
start a node 772 26
start a node 771 4
start a node 770 2
start a node 770 2
start a node 771 22
start a node 772 3
start a node 771 2
start a node 770 1
start a node 770 1
start a node 771 1
start a node 774 11
start a node 773 8
start a node 773 3
start a node 778 313
start a node 777 148
start a node 776 127
start a node 775 121
start a node 774 5
start a node 773 3
start a node 773 2
start a node 774 116
start a node 775 6
start a node 774 3
start a node 773 2
start a node 773 1
start a node 774 3
start a node 776 21
start a node 775 10
start a node 774 7
start a node 773 6
start a node 773 1
start a node 774 3
start a node 775 11
start a node 774 5
start a node 773 3
start 

start a node 773 2
start a node 774 12
start a node 773 8
start a node 773 4
start a node 775 21
start a node 774 2
start a node 774 19
start a node 776 189
start a node 775 149
start a node 774 43
start a node 773 38
start a node 772 19
start a node 771 16
start a node 771 3
start a node 770 1
start a node 770 2
start a node 772 19
start a node 771 14
start a node 770 12
start a node 770 2
start a node 771 5
start a node 770 4
start a node 770 1
start a node 773 5
start a node 772 4
start a node 772 1
start a node 774 106
start a node 773 74
start a node 772 64
start a node 772 10
start a node 771 8
start a node 770 1
start a node 770 7
start a node 771 2
start a node 773 32
start a node 772 6
start a node 771 5
start a node 771 1
start a node 772 26
start a node 771 10
start a node 770 5
start a node 769 3
start a node 769 2
start a node 768 1
start a node 768 1
start a node 770 5
start a node 771 16
start a node 770 14
start a node 769 2
start a node 768 1
start a node 768 1
start a

start a node 772 12
start a node 772 1
start a node 773 12
start a node 772 8
start a node 771 7
start a node 771 1
start a node 772 4
start a node 771 1
start a node 771 3
start a node 778 1298
start a node 777 1266
start a node 777 32
start a node 776 25
start a node 775 3
start a node 774 2
start a node 774 1
start a node 775 22
start a node 776 7
start a node 775 4
start a node 774 2
start a node 774 2
start a node 775 3
start a node 774 2
start a node 774 1
start a node 779 388
start a node 778 215
start a node 777 128
start a node 776 29
start a node 775 11
start a node 774 7
start a node 774 4
start a node 773 3
start a node 773 1
start a node 775 18
start a node 774 9
start a node 773 8
start a node 773 1
start a node 774 9
start a node 776 99
start a node 775 74
start a node 774 68
start a node 773 67
start a node 773 1
start a node 774 6
start a node 773 3
start a node 773 3
start a node 772 1
start a node 772 2
start a node 775 25
start a node 774 9
start a node 773 6
start 

...

start a node 774 14
start a node 773 11
start a node 773 3
start a node 772 2
start a node 771 1
start a node 771 1
start a node 772 1
start a node 774 2
start a node 778 712
start a node 777 386
start a node 776 118
start a node 775 90
start a node 774 80
start a node 773 78
start a node 773 2
start a node 772 1
start a node 772 1
start a node 774 10
start a node 773 3
start a node 773 7
start a node 772 4
start a node 772 3
start a node 775 28
start a node 774 13
start a node 773 2
start a node 772 1
start a node 772 1
start a node 773 11
start a node 772 10
start a node 772 1
start a node 774 15
start a node 773 7
start a node 772 4
start a node 772 3
start a node 773 8
start a node 772 1
start a node 772 7
start a node 776 268
start a node 775 204
start a node 774 45
start a node 773 41
start a node 772 37
start a node 771 36
start a node 770 35
start a node 770 1
start a node 771 1
start a node 772 4
start a node 771 2
start a node 770 1
start a node 770 1
start a node 771 2
start

start a node 770 4
start a node 770 1
start a node 772 32
start a node 771 28
start a node 770 22
start a node 770 6
start a node 769 3
start a node 769 3
start a node 771 4
start a node 770 3
start a node 770 1
start a node 774 37
start a node 773 35
start a node 772 4
start a node 771 2
start a node 771 2
start a node 770 1
start a node 770 1
start a node 772 31
start a node 773 2
start a node 772 1
start a node 772 1
start a node 779 322
start a node 778 190
start a node 777 163
start a node 776 154
start a node 776 9
start a node 775 4
start a node 774 2
start a node 774 2
start a node 773 1
start a node 773 1
start a node 775 5
start a node 774 4
start a node 774 1
start a node 777 27
start a node 776 17
start a node 775 6
start a node 775 11
start a node 774 2
start a node 774 9
start a node 773 2
start a node 772 1
start a node 772 1
start a node 773 7
start a node 776 10
start a node 778 132
start a node 777 95
start a node 776 84
start a node 775 6
start a node 774 3
start a n

start a node 770 5
start a node 770 1
start a node 774 92
start a node 773 51
start a node 772 39
start a node 771 24
start a node 770 4
start a node 769 3
start a node 769 1
start a node 770 20
start a node 771 15
start a node 770 8
start a node 770 7
start a node 769 2
start a node 768 1
start a node 768 1
start a node 769 5
start a node 772 12
start a node 771 3
start a node 770 2
start a node 769 1
start a node 769 1
start a node 770 1
start a node 771 9
start a node 773 41
start a node 772 2
start a node 771 1
start a node 771 1
start a node 772 39
start a node 771 38
start a node 771 1
start a node 776 125
start a node 775 67
start a node 774 29
start a node 773 26
start a node 772 1
start a node 772 25
start a node 773 3
start a node 772 2
start a node 772 1
start a node 774 38
start a node 773 6
start a node 772 3
start a node 771 2
start a node 771 1
start a node 772 3
start a node 771 2
start a node 771 1
start a node 773 32
start a node 772 31
start a node 771 2
start a node

start a node 768 3
start a node 768 1
start a node 770 10
start a node 771 2
start a node 772 1
start a node 773 44
start a node 772 43
start a node 772 1
start a node 774 5
start a node 773 1
start a node 773 4
start a node 777 102
start a node 776 33
start a node 775 24
start a node 774 9
start a node 774 15
start a node 773 5
start a node 772 4
start a node 772 1
start a node 773 10
start a node 772 7
start a node 771 6
start a node 771 1
start a node 772 3
start a node 771 1
start a node 771 2
start a node 775 9
start a node 774 4
start a node 773 3
start a node 773 1
start a node 774 5
start a node 776 69
start a node 775 13
start a node 774 11
start a node 774 2
start a node 773 1
start a node 773 1
start a node 775 56
start a node 774 29
start a node 773 16
start a node 773 13
start a node 772 8
start a node 771 2
start a node 770 1
start a node 770 1
start a node 771 6
start a node 772 5
start a node 774 27
start a node 773 7
start a node 772 3
start a node 771 2
start a node 7

start a node 776 22
start a node 775 13
start a node 774 8
start a node 773 7
start a node 773 1
start a node 774 5
start a node 773 2
start a node 772 1
start a node 772 1
start a node 773 3
start a node 775 9
start a node 774 3
start a node 774 6
start a node 773 3
start a node 773 3
start a node 776 687
start a node 778 579
start a node 777 198
start a node 776 61
start a node 775 7
start a node 774 3
start a node 773 2
start a node 772 1
start a node 772 1
start a node 773 1
start a node 774 4
start a node 775 54
start a node 776 137
start a node 775 104
start a node 774 88
start a node 773 11
start a node 772 3
start a node 771 1
start a node 771 2
start a node 772 8
start a node 771 3
start a node 770 2
start a node 770 1
start a node 771 5
start a node 773 77
start a node 772 71
start a node 771 69
start a node 770 2
start a node 769 1
start a node 769 1
start a node 770 67
start a node 769 1
start a node 769 66
start a node 771 2
start a node 772 6
start a node 774 16
start a n

In [60]:
def clas(testDataArr, tree):
    while True:
        (key,value), = tree.items()
        if type(value).__name__ =="dict":
            val = testDataArr[key]
            testDataArr=np.delete(testDataArr,key)
            tree = value[val]     
            #例如{403: {0: 7, 1: {297:7}}，dataVal=0
            if type(tree).__name__!="dict":
                return tree
        else:
            return value

def predict(testDataArr,tree):
    prelabels = []
    for i in range(testDataArr.shape[0]):
        dataArr = testDataArr[i].flatten()
        label = clas(dataArr,tree)
        prelabels.append(label)
    return prelabels

In [62]:
pre = np.array(predict(testDataArr,tree)).flatten()
print("正确率:",np.sum(pre==testLabelArr)/testLabelArr.size*100)

正确率: 88.09
